In [54]:
import eikon as ek
import pandas as pd

# Replace with your own App Key
ek.set_app_key('5d67b807cafe47c0acf2248d6f56ec409013c511')


In [51]:
import warnings
warnings.filterwarnings("ignore", message=".*errors='ignore'.*")


# Example: MSCI World constituents
CHAIN_RIC = "0#.SPX"

# Retrieve RICs from the index chain
constituents, err = ek.get_data(CHAIN_RIC, ["RIC", "CompanyName"])
if err:
    print("Error retrieving chain RIC constituents:", err)
else:
    print(f"Number of constituents retrieved: {len(constituents)}")
    print(constituents.head(10))

Error retrieving chain RIC constituents: [{'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'POOL.OQ'", 'row': 0}, {'code': 251658244, 'col': 2, 'message': "Error: Field 'COMPANYNAME' was not found in response for the instrument 'POOL.OQ'", 'row': 0}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'CHRW.OQ'", 'row': 1}, {'code': 251658244, 'col': 2, 'message': "Error: Field 'COMPANYNAME' was not found in response for the instrument 'CHRW.OQ'", 'row': 1}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'AJG.N'", 'row': 2}, {'code': 251658244, 'col': 2, 'message': "Error: Field 'COMPANYNAME' was not found in response for the instrument 'AJG.N'", 'row': 2}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'CNP.N'", 'row': 3}, {'code': 251658244, 'col': 2, 'messag

In [52]:
chains = ["0#.SPX", "0#.DJI", "0#.FTSE", "0#.STOXX50E", "0#.N225", "0#.HSI"]
for chain_ric in chains:
    df, err = ek.get_data(chain_ric, ["RIC", "CompanyName"])
    if err:
        print(f"{chain_ric} -> Error:", err)
    else:
        print(f"{chain_ric} -> Success, retrieved {len(df)} constituents")

0#.SPX -> Error: [{'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'POOL.OQ'", 'row': 0}, {'code': 251658244, 'col': 2, 'message': "Error: Field 'COMPANYNAME' was not found in response for the instrument 'POOL.OQ'", 'row': 0}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'CHRW.OQ'", 'row': 1}, {'code': 251658244, 'col': 2, 'message': "Error: Field 'COMPANYNAME' was not found in response for the instrument 'CHRW.OQ'", 'row': 1}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'AJG.N'", 'row': 2}, {'code': 251658244, 'col': 2, 'message': "Error: Field 'COMPANYNAME' was not found in response for the instrument 'AJG.N'", 'row': 2}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'RIC' was not found in response for the instrument 'CNP.N'", 'row': 3}, {'code': 251658244, 'col': 2, 'message': "Error: Field 'COMPA

In [55]:
# Step 1: Use a simpler set of fields
screen_expr = (
    'SCREEN('
    'U(IN(Equity(active,public,primary))/*UNV:Public*/),'
    'IN(TR.RegCountryCode,"US"),'
    'TR.TRESGScore(Period=FY0)>0,'
    'CURN=USD)'
)

universe_df, err = ek.get_data(
    screen_expr, 
    fields=["Instrument", "TR.CUSIP"]  # Use "Instrument" instead of "TR.PrimaryInstrument"
)

if err:
    print("Error:", err)
else:
    # Print out the columns to verify
    print("Columns in universe_df:", universe_df.columns)
    print(universe_df.head())

    # If "Instrument" is indeed present, do:
    ric_list = universe_df["Instrument"].dropna().unique().tolist()
    print("Found instruments:", len(ric_list))


Error: [{'code': 251658244, 'col': 1, 'message': "Error: Field 'INSTRUMENT' was not found in response for the instrument 'ECL.N'", 'row': 0}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'INSTRUMENT' was not found in response for the instrument 'STZ.N'", 'row': 1}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'INSTRUMENT' was not found in response for the instrument 'EA.OQ'", 'row': 2}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'INSTRUMENT' was not found in response for the instrument 'DHIL.OQ'", 'row': 3}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'INSTRUMENT' was not found in response for the instrument 'EMR.N'", 'row': 4}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'INSTRUMENT' was not found in response for the instrument 'EBF.N'", 'row': 5}, {'code': 251658244, 'col': 1, 'message': "Error: Field 'INSTRUMENT' was not found in response for the instrument 'EOG.N'", 'row': 6}, {'code': 251658244, 'col': 1, 'message': "Error: Fie

In [56]:
# Example chunked retrieval from 2012 to present, monthly frequency
FIELDS = [
    "TR.ClosePrice",
    "TR.MarketCap",
    "TR.Volume",
    "TR.TRESGScore",
    "TR.EnvironmentPillarScore",
    "TR.SocialPillarScore",
    "TR.CorporateGovernancePillarScore"
]
PARAMETERS = {
    "SDate": "2012-01-01",
    "EDate": "2023-12-31",
    "FRQ": "M"  # monthly
}

# chunker function:
def chunker(seq, size):
    for pos in range(0, len(seq), size):
        yield seq[pos:pos + size]

all_frames = []
for subset in chunker(ric_list, 200):
    df_chunk, err_chunk = ek.get_data(subset, fields=FIELDS, parameters=PARAMETERS)
    if err_chunk:
        print("Error in chunk:", err_chunk)
        continue
    all_frames.append(df_chunk)

combined_df = pd.concat(all_frames, ignore_index=True)
print(combined_df.head(50))
print("Total records:", len(combined_df))


KeyboardInterrupt: 

In [45]:
import eikon as ek
import pandas as pd

ek.set_app_key("5d67b807cafe47c0acf2248d6f56ec409013c511")


In [57]:

# 1) Screener expression with multiple countries
#    We filter by exchange country codes (TR.ExchangeCountryCode).
#    Also, TR.TRESGScore(Period=FY0) > 0 to ensure we only get companies with an ESG score.
screener_expr = """
SCREEN(
  U(IN(Equity(active,public,primary))/*UNV:Public*/),
  IN(TR.ExchangeCountryCode,"US","GB","JP","DE","FR","CA","AU","CN"),
  TR.TRESGScore(Period=FY0) > 0
)
"""

# 2) We request at least the RIC and a couple of quick reference fields (e.g., TR.CommonName, ESG Score).
fields_for_universe = [
    "TR.RIC",
    "TR.CommonName",
    "TR.TRESGScore"  # just to confirm coverage in the screener result
]

# 3) Retrieve the universe
df_universe, err = ek.get_data(screener_expr, fields_for_universe)

if err:
    raise RuntimeError(f"Screener error: {err}")

print("Columns returned:", df_universe.columns)
print("First 10 rows:\n", df_universe.head(10))
print("Total instruments found:", len(df_universe))

# 4) Extract RICs
ric_col = "RIC" if "RIC" in df_universe.columns else "Instrument"
ric_list = df_universe[ric_col].dropna().unique().tolist()

print("Number of unique RICs in the universe:", len(ric_list))


Columns returned: Index(['Instrument', 'RIC', 'Company Common Name', 'ESG Score'], dtype='object')
First 10 rows:
   Instrument      RIC             Company Common Name  ESG Score
0      KE.OQ    KE.OQ         Kimball Electronics Inc  60.279457
1       TX.N     TX.N                      Ternium SA  51.219531
2    MELI.OQ  MELI.OQ                MercadoLibre Inc  54.954479
3     CBA.AX   CBA.AX  Commonwealth Bank of Australia   89.05102
4     BKW.AX   BKW.AX                  Brickworks Ltd  54.386808
5     BOQ.AX   BOQ.AX          Bank of Queensland Ltd  69.802774
6     AOV.AX   AOV.AX                      Amotiv Ltd   30.17005
7     HVN.AX   HVN.AX      Harvey Norman Holdings Ltd  21.047588
8     MAH.AX   MAH.AX           Macmahon Holdings Ltd  40.662416
9     PMV.AX   PMV.AX         Premier Investments Ltd  41.575733
Total instruments found: 7235
Number of unique RICs in the universe: 7235


In [59]:
print(df_universe.head())

  Instrument      RIC             Company Common Name  ESG Score
0      KE.OQ    KE.OQ         Kimball Electronics Inc  60.279457
1       TX.N     TX.N                      Ternium SA  51.219531
2    MELI.OQ  MELI.OQ                MercadoLibre Inc  54.954479
3     CBA.AX   CBA.AX  Commonwealth Bank of Australia   89.05102
4     BKW.AX   BKW.AX                  Brickworks Ltd  54.386808


In [60]:
# Chunking function
def chunker(seq, size):
    for pos in range(0, len(seq), size):
        yield seq[pos:pos + size]

CHUNK_SIZE = 200

FIELDS = [
    "TR.CommonName",
    "TR.HeadquartersCountry",
    "TR.TRBCIndustry",
    "TR.MarketCap",
    "TR.Volume",
    "TR.ClosePrice",
    "TR.TRESGScore",
    "TR.EnvironmentPillarScore",
    "TR.SocialPillarScore",
    "TR.CorporateGovernancePillarScore",
    "TR.CO2Emissions"
]

PARAMS = {
    "SDate": "2012-01-01",
    "EDate": "2023-12-31",
    "FRQ": "M"
}

ric_list = df_universe["RIC"].tolist()

all_dfs = []
for i, subset in enumerate(chunker(ric_list, CHUNK_SIZE)):
    print(f"Processing chunk {i+1} with {len(subset)} RICs...")

    df_chunk, err_chunk = ek.get_data(
        instruments=subset,
        fields=FIELDS,
        parameters=PARAMS
    )

    if err_chunk:
        print(f"Error in chunk {i+1}:", err_chunk)
        continue

    all_dfs.append(df_chunk)

# Combine all chunks into one DataFrame
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
else:
    combined_df = pd.DataFrame()

# Rename for consistency with df_universe
combined_df.rename(columns={
    "Instrument": "RIC",
    "Common Name": "Company Common Name",
    "ESG Score": "ESG Score (Retrieved)"
}, inplace=True)

# Merge historical time series with static company info
df_final = pd.merge(
    combined_df,
    df_universe,
    on="RIC",
    how="left",
    suffixes=('', '_Original')
)

# Optional: drop or rearrange columns as needed
df_final = df_final[[
    'RIC', 'Company Common Name', 'ESG Score',
    'Date', 'Close Price', 'Volume', 'Market Cap',
    'Headquarters Country', 'Industry',
    'ESG Score (Retrieved)', 'Environmental Pillar Score',
    'Social Pillar Score', 'Corporate Governance Pillar Score',
    'CO2 Emissions'
]]

print(df_final.head())

Processing chunk 1 with 200 RICs...
Processing chunk 2 with 200 RICs...
Error in chunk 2: [{'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.Volume' and some specific identifier(s).", 'row': 288}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.CLOSEPRICE' and some specific identifier(s).", 'row': 288}, {'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.Volume' and some specific identifier(s).", 'row': 289}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.CLOSEPRICE' and some specific identifier(s).", 'row': 289}, {'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.Volume' and some specific identifier(s).", 'row': 290}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.CLOSEPRICE' and some specific identifier(s).", 'row': 290}, {'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.Volume' and some speci

C:\Users\jr24440\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\jr24440\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


Processing chunk 4 with 200 RICs...
Processing chunk 5 with 200 RICs...
Processing chunk 6 with 200 RICs...
Error in chunk 6: [{'code': 100, 'col': 1, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 2, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 3, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 4, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 5, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 6, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 7, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 8, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 9, 'message': 'Asynchronous Query library internal error.', 'row': 0}, {'code': 100, 'col': 10, 'message': 'Asynchro

KeyError: "['Date', 'Market Cap', 'Headquarters Country', 'Industry', 'Corporate Governance Pillar Score', 'CO2 Emissions'] not in index"

In [63]:
import eikon as ek
import pandas as pd

def chunker(seq, size):
    for pos in range(0, len(seq), size):
        yield seq[pos:pos + size]

CHUNK_SIZE = 100  # Conservative chunk size to avoid timeouts

FIELDS = [
    "TR.CommonName",
    "TR.HeadquartersCountry",
    "TR.TRBCIndustry",
    "TR.MarketCap",
    "TR.Volume",
    "TR.ClosePrice",
    "TR.TRESGScore",
    "TR.EnvironmentPillarScore",
    "TR.SocialPillarScore",
    "TR.CorporateGovernancePillarScore",
    "TR.CO2Emissions"
]

PARAMS = {
    "SDate": "2012-01-01",
    "EDate": "2023-12-31",
    "FRQ": "M"  # Monthly
}

ric_list = df_universe["RIC"].tolist()

all_dfs = []
for i, subset in enumerate(chunker(ric_list, CHUNK_SIZE)):
    print(f"Processing chunk {i+1} with {len(subset)} RICs...")

    df_chunk, err = ek.get_data(
        instruments=subset,
        fields=FIELDS,
        parameters=PARAMS
    )

    if err:
        print(f"Error in chunk {i+1}:", err)
        continue

    all_dfs.append(df_chunk)

# Combine everything
df_combined = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()

# Rename columns for consistency
df_combined.rename(columns={
    "Instrument": "RIC",
    "Date": "Date",
    "TR.CommonName": "Company Common Name",
    "TR.HeadquartersCountry": "Headquarters Country",
    "TR.TRBCIndustry": "Industry",
    "TR.MarketCap": "Market Cap",
    "TR.Volume": "Volume",
    "TR.ClosePrice": "Close Price",
    "TR.TRESGScore": "ESG Score (Retrieved)",
    "TR.EnvironmentPillarScore": "Environmental Pillar Score",
    "TR.SocialPillarScore": "Social Pillar Score",
    "TR.CorporateGovernancePillarScore": "Corporate Governance Pillar Score",
    "TR.CO2Emissions": "CO2 Emissions"
}, inplace=True)

# Merge with original ESG Score from your static df_universe
df_final = pd.merge(df_combined, df_universe[['RIC', 'ESG Score']], on="RIC", how="left")

# Order columns neatly
columns_order = [
    'RIC', 'Company Common Name', 'ESG Score', 'Date',
    'Close Price', 'Volume', 'Market Cap',
    'Headquarters Country', 'Industry',
    'ESG Score (Retrieved)', 'Environmental Pillar Score',
    'Social Pillar Score', 'Corporate Governance Pillar Score',
    'CO2 Emissions'
]

# Keep only existing columns to avoid KeyError
df_final = df_final[[col for col in columns_order if col in df_final.columns]]

print("\n✅ Final time-series DataFrame:")
print(df_final.head())


Processing chunk 1 with 100 RICs...
Processing chunk 2 with 100 RICs...
Processing chunk 3 with 100 RICs...
Error in chunk 3: [{'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.Volume' and some specific identifier(s).", 'row': 288}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.CLOSEPRICE' and some specific identifier(s).", 'row': 288}, {'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.Volume' and some specific identifier(s).", 'row': 289}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.CLOSEPRICE' and some specific identifier(s).", 'row': 289}, {'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.Volume' and some specific identifier(s).", 'row': 290}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.CLOSEPRICE' and some specific identifier(s).", 'row': 290}, {'code': 416, 'col': 4, 'message': "Unable to collect data for 

C:\Users\jr24440\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\jr24440\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


Processing chunk 6 with 100 RICs...
Processing chunk 7 with 100 RICs...
Processing chunk 8 with 100 RICs...
Processing chunk 9 with 100 RICs...
Processing chunk 10 with 100 RICs...
Processing chunk 11 with 100 RICs...
Processing chunk 12 with 100 RICs...
Processing chunk 13 with 100 RICs...
Processing chunk 14 with 100 RICs...
Processing chunk 15 with 100 RICs...
Processing chunk 16 with 100 RICs...
Processing chunk 17 with 100 RICs...
Processing chunk 18 with 100 RICs...
Processing chunk 19 with 100 RICs...
Processing chunk 20 with 100 RICs...
Processing chunk 21 with 100 RICs...
Processing chunk 22 with 100 RICs...
Processing chunk 23 with 100 RICs...
Processing chunk 24 with 100 RICs...
Processing chunk 25 with 100 RICs...
Processing chunk 26 with 100 RICs...
Processing chunk 27 with 100 RICs...
Processing chunk 28 with 100 RICs...
Processing chunk 29 with 100 RICs...
Processing chunk 30 with 100 RICs...
Processing chunk 31 with 100 RICs...
Processing chunk 32 with 100 RICs...
Proce

In [65]:
print(df_final.head())

     RIC      Company Common Name  Close Price  Volume  \
0  KE.OQ  Kimball Electronics Inc         11.0     0.0   
1  KE.OQ                                 12.02     0.0   
2  KE.OQ                                 10.18     0.0   
3  KE.OQ                                 12.52     0.0   
4  KE.OQ                                 14.14     0.0   

   Environmental Pillar Score  Social Pillar Score  
0                        <NA>                 <NA>  
1                        <NA>                 <NA>  
2                        <NA>                 <NA>  
3                        <NA>                 <NA>  
4                        <NA>                 <NA>  


In [64]:
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print("Columns returned by Eikon:", combined_df.columns.tolist())
else:
    combined_df = pd.DataFrame()
    print("No data retrieved.")

Columns returned by Eikon: ['Instrument', 'Company Common Name', 'Country of Headquarters', 'TRBC Industry Name', 'Volume', 'Close Price', 'ESG Score', 'Environmental Pillar Score', 'Social Pillar Score']


In [66]:
print(df_final.head())

     RIC      Company Common Name  Close Price  Volume  \
0  KE.OQ  Kimball Electronics Inc         11.0     0.0   
1  KE.OQ                                 12.02     0.0   
2  KE.OQ                                 10.18     0.0   
3  KE.OQ                                 12.52     0.0   
4  KE.OQ                                 14.14     0.0   

   Environmental Pillar Score  Social Pillar Score  
0                        <NA>                 <NA>  
1                        <NA>                 <NA>  
2                        <NA>                 <NA>  
3                        <NA>                 <NA>  
4                        <NA>                 <NA>  


In [35]:
combined_df = combined_df.dropna()

In [67]:
print(combined_df.head())

  Instrument      Company Common Name   Country of Headquarters  \
0      KE.OQ  Kimball Electronics Inc  United States of America   
1      KE.OQ                                                      
2      KE.OQ                                                      
3      KE.OQ                                                      
4      KE.OQ                                                      

  TRBC Industry Name  Volume  Close Price  ESG Score  \
0     Semiconductors     0.0         11.0       <NA>   
1                        0.0        12.02       <NA>   
2                        0.0        10.18       <NA>   
3                        0.0        12.52       <NA>   
4                        0.0        14.14       <NA>   

   Environmental Pillar Score  Social Pillar Score  
0                        <NA>                 <NA>  
1                        <NA>                 <NA>  
2                        <NA>                 <NA>  
3                        <NA>                 <N

In [47]:
print("Columns:", combined_df.columns)


Columns: Index(['Instrument', 'Company Common Name', 'Country of Headquarters',
       'Close Price', 'Volume', 'TRBC Industry Name', 'ESG Score',
       'Environmental Pillar Score', 'Social Pillar Score'],
      dtype='object')


In [70]:
import pandas as pd

# Assume combined_df is already loaded

# Step 1: Create per-instrument monthly dates
def generate_monthly_dates(group):
    group = group.copy()
    group['Date'] = pd.date_range(start='2012-01-01', periods=len(group), freq='MS')
    return group

combined_df = combined_df.groupby('Instrument', group_keys=False).apply(generate_monthly_dates)

# Step 2: Clean + fill metadata
fields_to_fill = ['Company Common Name', 'Country of Headquarters', 'TRBC Industry Name']

# Replace empty strings with NA so ffill works
combined_df[fields_to_fill] = combined_df[fields_to_fill].replace('', pd.NA)

# Forward-fill within each instrument group
combined_df[fields_to_fill] = combined_df.groupby('Instrument')[fields_to_fill].ffill()

# ✅ Optional: Reorder for readability
combined_df = combined_df[['Date', 'Instrument'] + fields_to_fill +
                          [col for col in combined_df.columns if col not in ['Date', 'Instrument'] + fields_to_fill]]

# Preview the fix
print(combined_df.head(10))


C:\Users\jr24440\AppData\Local\Temp\ipykernel_35236\1061285194.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = combined_df.groupby('Instrument', group_keys=False).apply(generate_monthly_dates)


        Date Instrument      Company Common Name   Country of Headquarters  \
0 2012-01-01      KE.OQ  Kimball Electronics Inc  United States of America   
1 2012-02-01      KE.OQ  Kimball Electronics Inc  United States of America   
2 2012-03-01      KE.OQ  Kimball Electronics Inc  United States of America   
3 2012-04-01      KE.OQ  Kimball Electronics Inc  United States of America   
4 2012-05-01      KE.OQ  Kimball Electronics Inc  United States of America   
5 2012-06-01      KE.OQ  Kimball Electronics Inc  United States of America   
6 2012-07-01      KE.OQ  Kimball Electronics Inc  United States of America   
7 2012-08-01      KE.OQ  Kimball Electronics Inc  United States of America   
8 2012-09-01      KE.OQ  Kimball Electronics Inc  United States of America   
9 2012-10-01      KE.OQ  Kimball Electronics Inc  United States of America   

  TRBC Industry Name  Volume  Close Price  ESG Score  \
0     Semiconductors     0.0         11.0       <NA>   
1     Semiconductors     0.0 

In [71]:
print(combined_df.describe())

                                Date           Volume       Close Price  \
count                        1012754        1006034.0          811757.0   
mean   2017-12-15 22:53:48.729582592   3193775.540225    1941327.561812   
min              2012-01-01 00:00:00              0.0          0.000001   
25%              2014-12-01 00:00:00              0.0             10.17   
50%              2017-12-01 00:00:00           7627.5             27.89   
75%              2020-12-01 00:00:00         410717.5              97.3   
max              2023-12-01 00:00:00     8826611382.0     84510000000.0   
std                              NaN  25184735.865534  288183827.708544   

       ESG Score  Environmental Pillar Score  Social Pillar Score  
count   528875.0                    528863.0             528863.0  
mean   42.032416                   34.309896            42.630863  
min     0.073196                         0.0             0.167672  
25%    25.350696                    7.527453        

In [72]:
# Number of unique companies
num_companies = combined_df['Company Common Name'].nunique()

# Full list of unique industries
unique_industries = combined_df['TRBC Industry Name'].dropna().unique()
num_industries = len(unique_industries)

print(f"✅ Number of unique companies: {num_companies}")
print(f"✅ Number of unique industries: {num_industries}")
print("\n🧾 Full list of industries:")
for industry in sorted(unique_industries):
    print("-", industry)


✅ Number of unique companies: 7034
✅ Number of unique industries: 140

🧾 Full list of industries:
- Advanced Medical Equipment & Technology
- Advertising & Marketing
- Aerospace & Defense
- Agricultural Chemicals
- Airlines
- Airport Operators & Services
- Aluminum
- Apparel & Accessories
- Apparel & Accessories Retailers
- Appliances, Tools & Housewares
- Auto & Truck Manufacturers
- Auto Vehicles, Parts & Service Retailers
- Auto, Truck & Motorcycle Parts
- Banks
- Biotechnology & Medical Research
- Blockchain & Cryptocurrency
- Brewers
- Broadcasting
- Business Support Services
- Business Support Supplies
- Casinos & Gaming
- Closed End Funds
- Coal
- Commercial Printing Services
- Commercial REITs
- Commodity Chemicals
- Communications & Networking
- Computer & Electronics Retailers
- Computer Hardware
- Construction & Engineering
- Construction Materials
- Construction Supplies & Fixtures
- Consumer Goods Conglomerates
- Consumer Lending
- Consumer Publishing
- Corporate Financial

In [74]:
# Export combined_df to Excel
combined_df.to_excel("2012_2023_US_ESG.xlsx", index=False)